In [1]:
!pip install -qU transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 89.5 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader


2025-07-23 08:47:57.000031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753260477.375571      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753260477.489782      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Tải tokenizer và mô hình E5
model_name = "intfloat/multilingual-e5-large"
model = SentenceTransformer(model_name)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [5]:
import wandb
wandb.login(key="ffd1f629ad757d71b34c5c5702eb813197ed4f0e")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
data_path = "/kaggle/input/data-for-bgem3/train_pairs.jsonl"

train_examples = []
with open(data_path, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        query_text = data["query"]
        positives = data.get("pos", [])
        negatives = data.get("neg", [])
        # Thêm tiền tố cho query
        query_input = "query: " + query_text

        # Tạo các triplet cho mỗi cặp pos-neg
        for pos_text in positives:
            pos_input = "passage: " + pos_text
            if negatives:
                # Kết hợp với mỗi negative (hoặc một vài negative tiêu biểu nếu quá nhiều)
                for neg_text in negatives:
                    neg_input = "passage: " + neg_text
                    train_examples.append(InputExample(texts=[query_input, pos_input, neg_input]))
            else:
                # Nếu không có negative nào (hiếm khi), có thể bỏ qua hoặc tự tạo negative giả
                continue

# Số lượng mẫu huấn luyện tạo được:
print("Tổng số mẫu (triplet) huấn luyện:", len(train_examples))


Tổng số mẫu (triplet) huấn luyện: 458780


In [30]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=1)


In [31]:
train_loss = losses.TripletLoss(model=model, distance_metric=losses.TripletDistanceMetric.EUCLIDEAN)


In [32]:
# Tính toán số warmup_steps (10% tổng steps)
training_steps = len(train_dataloader) * 3  # số batch * số epoch (giả sử 3 epoch)
warmup_steps = int(0.1 * training_steps)

# Fine-tune mô hình
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    optimizer_params={'lr': 2e-5},
    warmup_steps=warmup_steps,
    output_path="output/e5_finetuned_model",
    show_progress_bar=True,
    use_amp=True
)


OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 854.12 MiB is free. Process 2747 has 13.90 GiB memory in use. Of the allocated memory 13.45 GiB is allocated by PyTorch, and 260.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)